Appendix
* Target sentences: `results/FinancialPhraseBank_DSAll_k-folds/predictions_senti-dd.csv`
* Senti-DD Lexicon: `data/DS50_Entire/Senti-DD.csv`
* Loughran-McDonald Lexicon: `data/LM_Word_List.csv`

In [1]:
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()
import nltk
nltk.download('omw-1.4')
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from itertools import product

data_filepath ='results/FinancialPhraseBank_DSAll_k-folds/predictions_senti-dd.csv'
senti_dd_lexicon_filepath = 'data/DS50_Entire/Senti-DD.csv'
lm_filepath = 'data/LM_Word_List.csv'

save_filepath = 'results/senti_dd_interpretability_evidence_for_appendix.csv'

C:\Users\Jihye Park\anaconda3\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package omw-1.4 to C:\Users\Jihye
[nltk_data]     Park\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
df = pd.read_csv(data_filepath)[['headline', 'label']]
lm_df = pd.read_csv(lm_filepath)
senti_dd_lexicon_df = pd.read_csv(senti_dd_lexicon_filepath)

print('sentences\n', df.head(), '\n')
print('Loughran-McDonald\n', lm_df.head(), '\n')
print('Senti-DD\n', senti_dd_lexicon_df.head(), '\n')

sentences
                                             headline     label
0  finnish developer and manufacturer of mobile p...  negative
1  the transaction is expected to be finalized by...   neutral
2  okmetic board of directors has also decided on...   neutral
3  indigo and somoncom serve 377 000 subscribers ...   neutral
4  to see a slide show of all the newest product ...   neutral 

Loughran-McDonald
            word     label
0       abandon  negative
1     abandoned  negative
2    abandoning  negative
3   abandonment  negative
4  abandonments  negative 

Senti-DD
   sentiment   entity directional_word
0  positive  quarter          acceler
1  positive  quarter           advanc
2  positive  quarter            award
3  positive  quarter           better
4  positive  quarter            climb 



In [3]:
def get_intersection(list_one, list_two):
    return list(set(list_one) & set(list_two))

lm_positive_words = lm_df[lm_df['label']=='positive']['word'].values
def detect_lm_positive_words(text):
    tokens = word_tokenize(text)
    return get_intersection(tokens, lm_positive_words)

lm_negative_words = lm_df[lm_df['label']=='negative']['word'].values
def detect_lm_negative_words(text):
    tokens = word_tokenize(text)
    return get_intersection(tokens, lm_negative_words)

lemmatizer=WordNetLemmatizer()
entity_list = list(senti_dd_lexicon_df['entity'].unique())
def detect_entities(text):
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in word_tokenize(text)]
    return get_intersection(lemmatized_tokens, entity_list)

stemmer = PorterStemmer()
directional_word_list = list(senti_dd_lexicon_df['directional_word'].unique())
def detect_directional_words(text):
    stemmed_tokens = [stemmer.stem(token) for token in word_tokenize(text)]
    return get_intersection(stemmed_tokens, directional_word_list)

senti_dd_positive_pair_list = ['_'.join((item1, item2)) for item1, item2 in list(senti_dd_lexicon_df[(senti_dd_lexicon_df['sentiment']=='positive')][['entity', 'directional_word']].values)]
def get_positive_pairs(entities, directional_words):
    this_pair_list = ['_'.join((item1, item2)) for item1, item2 in list(product(entities, directional_words))]
    return get_intersection(this_pair_list, senti_dd_positive_pair_list)

senti_dd_negative_pair_list = ['_'.join((item1, item2)) for item1, item2 in list(senti_dd_lexicon_df[(senti_dd_lexicon_df['sentiment']=='negative')][['entity', 'directional_word']].values)]
def get_negative_pairs(entities, directional_words):
    this_pair_list = ['_'.join((item1, item2)) for item1, item2 in list(product(entities, directional_words))]
    return get_intersection(this_pair_list, senti_dd_negative_pair_list)

def score_to_label(score):
    if score > 0: return 'positive'
    elif score < 0: return 'negative'
    else: return 'neutral'

df['lm_positive_words'] = df['headline'].progress_apply(lambda x: detect_lm_positive_words(x))
df['lm_negative_words'] = df['headline'].progress_apply(lambda x: detect_lm_negative_words(x))
df['detected_entities'] = df['headline'].progress_apply(lambda x: detect_entities(x))
df['detected_directional_words'] = df['headline'].progress_apply(lambda x: detect_directional_words(x))
df['senti_dd_pos_pairs'] = df.progress_apply(lambda x: get_positive_pairs(x['detected_entities'], x['detected_directional_words']), axis=1)
df['senti_dd_neg_pairs'] = df.progress_apply(lambda x: get_negative_pairs(x['detected_entities'], x['detected_directional_words']), axis=1)

df['senti_dd_score'] = df.progress_apply(lambda x: len(x['senti_dd_pos_pairs']) - len(x['senti_dd_neg_pairs']), axis=1)
df['senti_dd_prediction'] = df['senti_dd_score'].progress_apply(lambda x: score_to_label(x))
df['correct'] = df.apply(lambda x: x['label']==x['senti_dd_prediction'], axis=1)
df.head()

100%|██████████████████████████████████████████████████████████████████████████| 2259/2259 [00:00<00:00, 177015.52it/s]


,headline,label,lm_positive_words,lm_negative_words,detected_entities,detected_directional_words,senti_dd_pos_pairs,senti_dd_neg_pairs,senti_dd_score,senti_dd_prediction,correct
0,finnish developer and manufacturer of mobile p...,negative,[],[],"[manufacturer, sale, year, phone, net]",[],[],[],0,neutral,False
1,the transaction is expected to be finalized by...,neutral,[],[],[],[],[],[],0,neutral,True
2,okmetic board of directors has also decided on...,neutral,[],[],"[company, management, share]",[],[],[],0,neutral,True
3,indigo and somoncom serve 377 000 subscribers ...,neutral,[],[],[share],[],[],[],0,neutral,True
4,to see a slide show of all the newest product ...,neutral,[],[],[product],[],[],[],0,neutral,True


Save

In [4]:
df.to_csv(save_filepath, index=False)
print('Created {}'.format(save_filepath))

Created results/senti_dd_interpretability_evidence_for_appendix.csv


In [5]:
appendix_df = df[(df['detected_entities'].apply(lambda x: len(x)!=0)) & \
                 (df['detected_directional_words'].apply(lambda x: len(x)!=0)) & \
                 (df['lm_positive_words'].apply(lambda x: len(x)==0))& \
                 (df['lm_negative_words'].apply(lambda x: len(x)==0))]

result = pd.concat([appendix_df[appendix_df['correct']==True].sample(10), \
                   appendix_df[appendix_df['correct']==False].sample(10)])[['headline','label', 'senti_dd_prediction', 'senti_dd_pos_pairs', 'senti_dd_neg_pairs']]

result

,headline,label,senti_dd_prediction,senti_dd_pos_pairs,senti_dd_neg_pairs
339,diluted earnings per share eps rose to eur 3 6...,positive,positive,"[eur_rose, share_rose]",[]
926,comparable operating profit decreased to eur 1...,negative,negative,[],"[profit_decreas, eur_decreas, period_decreas]"
930,the world s second largest stainless steel mak...,positive,positive,"[maker_surg, period_surg, world_surg, net_surg...",[]
2190,net profit fell by almost half to +é 5 5 milli...,negative,negative,[],"[end_fell, net_fell, profit_fell]"
2165,in december alone the members of the lithuania...,positive,positive,"[percent_increas, beer_increas, sale_increas, ...",[]
22,that would be an increase from estimated sales...,positive,positive,"[year_increas, sale_increas]",[]
1557,ragutis which is controlled by the finnish bre...,positive,positive,"[beer_rise, share_rise, cent_rise, brewery_ris...",[]
2089,operating profit rose from eur 1 94 mn to eur ...,positive,positive,"[eur_rose, profit_rose]",[]
1917,both operating profit and net sales for the 12...,positive,positive,"[sale_increas, period_increas, net_increas, mo...",[]
1192,finnish silicon wafer technology company okmet...,positive,positive,"[period_up, profit_up, oyj_up, june_up]",[company_up]


In [6]:
save_result_filepath = save_filepath.replace('.csv', '_{}.csv'.format(len(result)))
result.to_csv(save_result_filepath, index=False)
print('Created {}'.format(save_result_filepath))

Created results/senti_dd_interpretability_evidence_for_appendix_20.csv
